In [2]:
%matplotlib inline
from keras.optimizers import SGD
import matplotlib.pyplot as plt
from tqdm import tqdm_notebook as tq
from PIL import Image
import keras
import pickle
from keras.models import load_model
from keras.models import Sequential
from keras.models import Model
from keras.utils.layer_utils import layer_from_config
from keras.layers import *
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from keras.preprocessing.image import load_img, img_to_array, random_rotation, flip_axis, array_to_img
import itertools
import gc
from keras.applications.vgg16 import VGG16
from keras.applications.vgg19 import VGG19
from keras.applications.resnet50 import ResNet50
from keras.preprocessing import image
from keras.applications.inception_v3 import preprocess_input as preprocess_input_inception
from keras.models import model_from_json
from keras.applications.inception_v3 import InceptionV3
from heatmap import to_heatmap
from heatmap import synset_to_dfs_ids
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from keras.preprocessing import image
import scipy
from heatmaps_generator import *
import os
import glob
import cv2
# To see the training error in real time.
from keras import callbacks
remote = callbacks.RemoteMonitor(root='http://localhost:9000')

Using Theano backend.


In [7]:
paths = get_all_images(regex = "./heatmaps/pickle/*/*/*/*.jpg")
paths[0]

'./heatmaps/pickle/ResNet50/horizontal flip/ALB/img_00003.jpg'

In [8]:
paths[0].split("/")[5]

'ALB'

In [9]:
dic_paths = {}
for path in paths:
    key = path.split("/")[3] + path.split("/")[4]
    add_to_dic_list(key, path, dic_paths)
    
for key, value in dic_paths.items():
    print(key + ": " + str(len(value)))
    
nb_samples = 1000000
keys_to_delete=[]
for key, value in dic_paths.items():
    string = key + " has " + str(len(value)) +" in it. Delete? [Y/nothing]"
    answer = input(string)
    if answer != "":
        keys_to_delete.append(key)
    else:
        nb_samples = min(nb_samples,len(value))

for key in keys_to_delete:
    dic_paths.pop(key)

ResNet50resolution 25 down: 180
ResNet50resolution 25 up: 180
ResNet50identity: 181
ResNet50vertical flip: 181
VGG16horizontal flip: 126
VGG16identity: 127
ResNet50horizontal flip: 181
ResNet50resolution 25 down has 180 in it. Delete? [Y/nothing]
ResNet50resolution 25 up has 180 in it. Delete? [Y/nothing]
ResNet50identity has 181 in it. Delete? [Y/nothing]
ResNet50vertical flip has 181 in it. Delete? [Y/nothing]
VGG16horizontal flip has 126 in it. Delete? [Y/nothing]
VGG16identity has 127 in it. Delete? [Y/nothing]
ResNet50horizontal flip has 181 in it. Delete? [Y/nothing]


In [10]:
t = tq(total=nb_samples)
heatmaps = []
list_transformations = []
masks = []
done = False # If we've already loaded the masks
for key, value in dic_paths.items():
    list_transformations.append(key)
    h = []
    for path in value[:nb_samples]:
        if not done:
            t.update()
            masks.append(get_mask_from_img(path.split("/")[5],path.split("/")[6], resolution("train/"+path.split("/")[5]+"/" + path.split("/")[6])))
        with open(path, "rb") as f:
            h.append(pickle.load(f))
    heatmaps.append(h)
    done = True
t.close()

X = np.swapaxes(np.array(heatmaps), 0,1)
Y = np.expand_dims(np.array(masks), axis=1)

In [11]:
list_transformations

['ResNet50resolution 25 down',
 'ResNet50resolution 25 up',
 'ResNet50identity',
 'ResNet50vertical flip',
 'VGG16horizontal flip',
 'VGG16identity',
 'ResNet50horizontal flip']

In [12]:
print(X.shape)
print(Y.shape)

(126, 7, 100, 100)
(126, 1, 100, 100)


In [14]:
Z = np.mean(X, axis=1)
#Z = np.expand_dims(Z, axis=1)
Z.shape

(126, 100, 100)